# 要約 
このJupyter Notebookは、Kaggleの「LMSYS - Chatbot Arena」コンペティションにおいて、人間による好み予測に関連した問題に取り組んでいます。具体的には、大規模言語モデル（LLM）が生成したチャットボットの応答の中から、どちらの応答が人間に好まれるかを予測するための機械学習モデルを構築しています。

### 問題の背景
コンペティションの課題は、二つのチャットボットが提供する応答に対するユーザーの好みを予測することです。このため、提供されたデータセットから学習し、テストセットに対する予測を行う必要があります。

### 使用されている手法とライブラリ
1. **ライブラリのインストール**: コンピュータビジョンライブラリや機械学習、トランスフォーマーモデルのサポートをするため、以下のライブラリを使用しています。
   - `accelerate`, `aiohttp`, `bitsandbytes`, `triton`, `unsloth`, `peft`, `trl`, `transformers`, `datasets`など。
   
2. **PyTorch**及びCUDAの使用: GPU上での高速な計算を実現するために、NVIDIAのCUDA関連のライブラリもインストールされています。PyTorchフレームワークを使用して、深層学習モデルを実装しています。

3. **データ前処理**: データセットの読み込みと処理が行われ、プロンプト生成関数が定義されています。訓練データ、評価データ、テストデータが生成され、モデルに渡されます。

4. **モデルの設定**: トークナイザーとモデルが初期化され、LoRA（Low-Rank Adaptation）を通じて効率的な微調整が行われます。

5. **学習プロセス**: `SFTTrainer`クラスが用いられ、ハイパーパラメータやトレーニング戦略が設定され、モデルの訓練が行われます。

6. **推論と評価**: 学習後、テストデータに対する応答の予測が行われ、それぞれの予測結果をもとに評価関数が実行されます。また、ロジット（各モデルの応答に対する確率予測）の抽出も行われ、最終的に結果をCSVファイルとして出力する準備が整います。

このNotebookは、データの前処理からモデルの構築、学習、評価、そして最終的な予測結果の出力までを包含しており、機械学習フレームワークや自然言語処理（NLP）技術を幅広く実践しています。

---


# 用語概説 
以下は、Jupyter Notebookの内容から、機械学習・深層学習の初心者がつまずきそうな専門用語の簡単な解説です。

### 専門用語解説

1. **量子化 (Quantization)**:
   - 処理を高速化し、必要なメモリを減らすために、モデルの浮動小数点数（通常は32ビット）を低いビット数（例えば、4ビットや8ビット）に変換する手法。これにより、モデルサイズが小さくなり、リソースの少ない環境でも実行できる。

2. **PEFT (Parameter-Efficient Fine-Tuning)**:
   - 大きなモデルのパラメータ（学習可能な重み）の一部だけを更新する方法で、少量のデータで効果的に微調整を行うことができる技術。これにより、少ないリソースで特定のタスクに対するモデルの性能を向上させることができる。

3. **LoRA (Low-Rank Adaptation)**:
   - 微調整の一つの手法で、モデルの重みに対して低ランクな近似を使うことによって、パラメータの数を減らしつつ、モデルの性能を維持または向上させる。特にメモリ効率が良く、大規模モデルに対して使われることが多い。

4. **SDP (Scalable Data Parallel)**:
   - ディープラーニングのパラレルオプティマイゼーション手法の一つであり、モデルのトレーニングをスケールさせて効率よく行うことを目的としています。大規模データやモデルでのトレーニングにおいて特に効果を発揮します。

5. **Gradient Checkpointing**:
   - メモリ使用量を低減しながら大規模なモデルのトレーニングを行うための手法。通常のバックプロパゲーションでは全ての中間計算を保存する必要がありますが、グラデーションチェックポイントでは計算を部分的にキャッシュし、必要に応じて再計算することで消費メモリを削減する。

6. **ロジット (Logits)**:
   - 出力層から得られる生のスコアや信頼度のこと。特に分類タスクにおいては、事前処理なしの出力であり、通常、ソフトマックス関数を通じてクラス確率に変換される。例えば、モデルが各クラスに対する信頼の数値を示す。

7. **Triton**:
   - NVIDIAが提供するディープラーニングのライブラリで、高速化されたカスタムカーネルを用い、利用可能なハードウェア資源を最大限に活用して計算を行う。これにより、トレーニングや推論のパフォーマンスが向上する。

8. **バイアス (Bias)**:
   - モデルの予測において、システム的な誤差を指す。これは教育データの偏りや、データ収集方法によって引き起こされる可能性があります。バイアスは、モデルの予測が特定のクラスに対して過小評価または過大評価される原因となる。

9. **NCCL (NVIDIA Collective Communications Library)**:
   - GPU間でのデータ通信を最適化するためのライブラリ。特に分散トレーニングにおいて複数のGPUを効率的に利用するための通信手法を提供します。

10. **フラッシュSDP (Flash SDP)**:
    - メモリ効率とパフォーマンスの向上を図った特定のデータパラレル実行戦略。GPUのメモリを効率的に使用しながら高いトレーニングスループットを実現します。

これらの用語は、実務経験が少ない初心者にとっては馴染みが薄い可能性があり、理解することがモデルのトレーニングや推論の際に重要となります。

---


<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/accelerate-0.29.3-py3-none-any.whl'
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/aiohttp-3.9.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl'
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/fsspec-2024.3.1-py3-none-any.whl'
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl'
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/unsloth-2024.4-py3-none-any.whl'
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/peft-0.10.0-py3-none-any.whl'

```

</div>
<div class="column-right">

# 日本語訳

```python
# 必要なライブラリをインストールします
# accelerateパッケージをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/accelerate-0.29.3-py3-none-any.whl'
# aiohttpライブラリをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/aiohttp-3.9.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'
# bitsandbytesライブラリをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl'
# fsspecライブラリをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/fsspec-2024.3.1-py3-none-any.whl'
# tritonライブラリをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl'
# unslothライブラリをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/unsloth-2024.4-py3-none-any.whl'
# peftライブラリをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/peft-0.10.0-py3-none-any.whl'
```

</div>
</details>

In [ ]:
# 必要なライブラリをインストールします
# accelerateパッケージをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/accelerate-0.29.3-py3-none-any.whl'
# aiohttpライブラリをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/aiohttp-3.9.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'
# bitsandbytesライブラリをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl'
# fsspecライブラリをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/fsspec-2024.3.1-py3-none-any.whl'
# tritonライブラリをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl'
# unslothライブラリをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/unsloth-2024.4-py3-none-any.whl'
# peftライブラリをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/peft-0.10.0-py3-none-any.whl'

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

!pip install '/kaggle/input/llm-detect-pip/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_nvjitlink_cu12-12.3.52-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl'
##!pip install '/kaggle/input/llm-detect-pip/nvidia_nccl_cu12-2.18.1-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/nvidia-nccl-2-19-3/nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl'
#!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/triton-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'


```

</div>
<div class="column-right">

# 日本語訳

```python
# NVIDIAのCUDA関連ライブラリをインストールします
# nvrtcライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl'
# runtimeライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl'
# cuptiライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl'
# cublasライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl'
# cudnnライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl'
# cufftライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl'
# curandライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl'
# nvjitlinkライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_nvjitlink_cu12-12.3.52-py3-none-manylinux1_x86_64.whl'
# cusparseライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl'
# cusolverライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl'
# nvtxライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl'
# NCCLライブラリをインストール（コメントアウトされています）
#!pip install '/kaggle/input/nvidia-nccl-2-19-3/nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl'
# Tritonライブラリをインストール（コメントアウトされています）
#!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/triton-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'
```

</div>
</details>

In [ ]:
# NVIDIAのCUDA関連ライブラリをインストールします
# nvrtcライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl'
# runtimeライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl'
# cuptiライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl'
# cublasライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl'
# cudnnライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl'
# cufftライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl'
# curandライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl'
# nvjitlinkライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_nvjitlink_cu12-12.3.52-py3-none-manylinux1_x86_64.whl'
# cusparseライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl'
# cusolverライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl'
# nvtxライブラリをインストール
!pip install '/kaggle/input/llm-detect-pip/nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl'
# NCCLライブラリをインストール（コメントアウトされています）
#!pip install '/kaggle/input/nvidia-nccl-2-19-3/nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl'
# Tritonライブラリをインストール（コメントアウトされています）
#!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/triton-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/shtab-1.7.1-py3-none-any.whl'
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/tyro-0.8.3-py3-none-any.whl'
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/trl-0.8.5-py3-none-any.whl'
```

</div>
<div class="column-right">

# 日本語訳

```python
# 必要なパッケージをインストールします
# shtabライブラリをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/shtab-1.7.1-py3-none-any.whl'
# tyroライブラリをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/tyro-0.8.3-py3-none-any.whl'
# trlライブラリをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/trl-0.8.5-py3-none-any.whl'
```

</div>
</details>

In [ ]:
# 必要なパッケージをインストールします
# shtabライブラリをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/shtab-1.7.1-py3-none-any.whl'
# tyroライブラリをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/tyro-0.8.3-py3-none-any.whl'
# trlライブラリをインストール
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/trl-0.8.5-py3-none-any.whl'

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/nvidia_nccl_cu12-2.18.1-py3-none-manylinux1_x86_64.whl'
```

</div>
<div class="column-right">

# 日本語訳

```python
# NVIDIAのNCCLライブラリをインストールします
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/nvidia_nccl_cu12-2.18.1-py3-none-manylinux1_x86_64.whl'
```

</div>
</details>

In [ ]:
# NVIDIAのNCCLライブラリをインストールします
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/nvidia_nccl_cu12-2.18.1-py3-none-manylinux1_x86_64.whl'

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl'
```

</div>
<div class="column-right">

# 日本語訳

```python
# Tritonライブラリをインストールします
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl'
```

</div>
</details>

In [ ]:
# Tritonライブラリをインストールします
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl'

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl'
```

</div>
<div class="column-right">

# 日本語訳

```python
# PyTorchライブラリをインストールします
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl'
```

</div>
</details>

In [ ]:
# PyTorchライブラリをインストールします
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl'

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/xformers-0.0.22.post7-cp310-cp310-manylinux2014_x86_64.whl'
```

</div>
<div class="column-right">

# 日本語訳

```python
# xformersライブラリをインストールします
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/xformers-0.0.22.post7-cp310-cp310-manylinux2014_x86_64.whl'
```

</div>
</details>

In [ ]:
# xformersライブラリをインストールします
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/xformers-0.0.22.post7-cp310-cp310-manylinux2014_x86_64.whl'

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import torch
print(f"pytorch version {torch.__version__}")
```

</div>
<div class="column-right">

# 日本語訳

```python
# torchライブラリをインポートします
import torch
# 現在のPyTorchのバージョンを表示します
print(f"pytorch version {torch.__version__}")  # PyTorchのバージョンを出力します
```

</div>
</details>

In [ ]:
# torchライブラリをインポートします
import torch
# 現在のPyTorchのバージョンを表示します
print(f"pytorch version {torch.__version__}")  # PyTorchのバージョンを出力します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

```

</div>
<div class="column-right">

# 日本語訳

```python
# このセルは空です。必要なコードをここに追加してください。
```

</div>
</details>

In [ ]:
# このセルは空です。必要なコードをここに追加してください。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

```

</div>
<div class="column-right">

# 日本語訳

```python
# このセルは空です。必要なコードをここに追加してください。
```

</div>
</details>

In [ ]:
# このセルは空です。必要なコードをここに追加してください。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from unsloth import FastLanguageModel
```

</div>
<div class="column-right">

# 日本語訳

```python
# unslothライブラリからFastLanguageModelクラスをインポートします
from unsloth import FastLanguageModel  # 効率的な言語モデルを使用するためのクラスです
```

</div>
</details>

In [ ]:
# unslothライブラリからFastLanguageModelクラスをインポートします
from unsloth import FastLanguageModel  # 効率的な言語モデルを使用するためのクラスです

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

```

</div>
<div class="column-right">

# 日本語訳

```python
# このセルは空です。必要なコードをここに追加してください。
```

</div>
</details>

In [ ]:
# このセルは空です。必要なコードをここに追加してください。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

```

</div>
<div class="column-right">

# 日本語訳

```python
# このセルは空です。必要なコードをここに追加してください。
```

</div>
</details>

In [ ]:
# このセルは空です。必要なコードをここに追加してください。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

```

</div>
<div class="column-right">

# 日本語訳

```python
# このセルは空です。必要なコードをここに追加してください。
```

</div>
</details>

In [ ]:
# このセルは空です。必要なコードをここに追加してください。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
```

</div>
<div class="column-right">

# 日本語訳

```python
# CUDAのメモリ効率的なSDP（Scalable Data Parallel）を無効にします
torch.backends.cuda.enable_mem_efficient_sdp(False)  # メモリ効率的SDPを無効にします
# CUDAのフラッシュSDPを無効にします
torch.backends.cuda.enable_flash_sdp(False)  # フラッシュSDPを無効にします
```

</div>
</details>

In [ ]:
# CUDAのメモリ効率的なSDP（Scalable Data Parallel）を無効にします
torch.backends.cuda.enable_mem_efficient_sdp(False)  # メモリ効率的SDPを無効にします
# CUDAのフラッシュSDPを無効にします
torch.backends.cuda.enable_flash_sdp(False)  # フラッシュSDPを無効にします

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import pandas as pd
import numpy as np
```

</div>
<div class="column-right">

# 日本語訳

```python
# pandasライブラリをインポートします
import pandas as pd  # データ操作と分析を行うためのライブラリ
# numpyライブラリをインポートします
import numpy as np  # 配列操作や数値計算のためのライブラリ
```

</div>
</details>

In [ ]:
# pandasライブラリをインポートします
import pandas as pd  # データ操作と分析を行うためのライブラリ
# numpyライブラリをインポートします
import numpy as np  # 配列操作や数値計算のためのライブラリ

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from peft import PeftModel
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split
```

</div>
<div class="column-right">

# 日本語訳

```python
# 必要なライブラリをインポートします
import os  # オペレーティングシステムに対する操作を行うためのライブラリ
from tqdm import tqdm  # 進捗バーを表示するためのライブラリ
import bitsandbytes as bnb  # メモリ効率を考慮したデータ型を扱うライブラリ
import torch  # PyTorchライブラリをインポート
import torch.nn as nn  # ニューラルネットワーク関連のモジュールを使用
import transformers  # Hugging Faceのtransformersライブラリをインポート
from datasets import Dataset  # データセットを扱うためのモジュール
from peft import LoraConfig, PeftConfig  # PEFT関連の設定をインポート
from peft import PeftModel  # PEFTモデルをインポート
from trl import SFTTrainer  # SFTTrainerをインポート
from trl import setup_chat_format  # チャットフォーマットを設定する関数をインポート
from transformers import (  # transformersライブラリから必要なコンポーネントをインポート
    AutoModelForCausalLM,  # 自己回帰型言語モデルのための自動モデル
    AutoTokenizer,  # 自動的にトークナイザーを取得するクラス
    BitsAndBytesConfig,  # メモリ効率の良い設定用のクラス
    TrainingArguments,  # 学習時の引数を定義するためのクラス
    pipeline,  # トークナイザーやモデルを簡単に使用するためのパイプライン
    logging  # ログ出力を扱うためのモジュール
)
from sklearn.metrics import (  # sklearnからメトリクスをインポート
    accuracy_score,  # 正確度を計算するための関数
    classification_report,  # 分類結果のレポートを生成するための関数
    confusion_matrix  # 混同行列を生成するための関数
)
from sklearn.model_selection import train_test_split  # トレーニングとテストのデータ分割を行うための関数
```

</div>
</details>

In [ ]:
# 必要なライブラリをインポートします
import os  # オペレーティングシステムに対する操作を行うためのライブラリ
from tqdm import tqdm  # 進捗バーを表示するためのライブラリ
import bitsandbytes as bnb  # メモリ効率を考慮したデータ型を扱うライブラリ
import torch  # PyTorchライブラリをインポート
import torch.nn as nn  # ニューラルネットワーク関連のモジュールを使用
import transformers  # Hugging Faceのtransformersライブラリをインポート
from datasets import Dataset  # データセットを扱うためのモジュール
from peft import LoraConfig, PeftConfig  # PEFT関連の設定をインポート
from peft import PeftModel  # PEFTモデルをインポート
from trl import SFTTrainer  # SFTTrainerをインポート
from trl import setup_chat_format  # チャットフォーマットを設定する関数をインポート
from transformers import (  # transformersライブラリから必要なコンポーネントをインポート
    AutoModelForCausalLM,  # 自己回帰型言語モデルのための自動モデル
    AutoTokenizer,  # 自動的にトークナイザーを取得するクラス
    BitsAndBytesConfig,  # メモリ効率の良い設定用のクラス
    TrainingArguments,  # 学習時の引数を定義するためのクラス
    pipeline,  # トークナイザーやモデルを簡単に使用するためのパイプライン
    logging  # ログ出力を扱うためのモジュール
)
from sklearn.metrics import (  # sklearnからメトリクスをインポート
    accuracy_score,  # 正確度を計算するための関数
    classification_report,  # 分類結果のレポートを生成するための関数
    confusion_matrix  # 混同行列を生成するための関数
)
from sklearn.model_selection import train_test_split  # トレーニングとテストのデータ分割を行うための関数

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
test = pd.read_csv(open('/kaggle/input/lmsys-chatbot-arena/test.csv'))
```

</div>
<div class="column-right">

# 日本語訳

```python
# テストデータセットを読み込みます
test = pd.read_csv(open('/kaggle/input/lmsys-chatbot-arena/test.csv'))  # CSVファイルを読み込み、DataFrameに格納します
```

</div>
</details>

In [ ]:
# テストデータセットを読み込みます
test = pd.read_csv(open('/kaggle/input/lmsys-chatbot-arena/test.csv'))  # CSVファイルを読み込み、DataFrameに格納します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def generate_test_prompt(data_point):
    return f"""Analyze the conversation between two chatbots (model_a and model_b) and their corresponding responses (response_a and response_b) to a given prompt. Determine which model provided the more preferred 
response based on the human preference label (Preference). Return the predicted preference as one of three labels: 'winner_model_a', 'winner_model_b', or 'winner_tie', along with the logits for each label.

Prompt: {data_point["prompt"]}
Model A Response: {data_point["response_a"]}
Model B Response: {data_point["response_b"]}
"""
```

</div>
<div class="column-right">

# 日本語訳

```python
# テストプロンプトを生成する関数を定義します
def generate_test_prompt(data_point):
    # 与えられたデータポイントに基づいて、会話を分析するプロンプトを生成します
    return f"""Analyze the conversation between two chatbots (model_a and model_b) and their corresponding responses (response_a and response_b) to a given prompt. Determine which model provided the more preferred 
response based on the human preference label (Preference). Return the predicted preference as one of three labels: 'winner_model_a', 'winner_model_b', or 'winner_tie', along with the logits for each label.

Prompt: {data_point["prompt"]}  # プロンプトを表示します
Model A Response: {data_point["response_a"]}  # モデルAの応答を表示します
Model B Response: {data_point["response_b"]}  # モデルBの応答を表示します
"""
```

</div>
</details>

In [ ]:
# テストプロンプトを生成する関数を定義します
def generate_test_prompt(data_point):
    # 与えられたデータポイントに基づいて、会話を分析するプロンプトを生成します
    return f"""Analyze the conversation between two chatbots (model_a and model_b) and their corresponding responses (response_a and response_b) to a given prompt. Determine which model provided the more preferred 
response based on the human preference label (Preference). Return the predicted preference as one of three labels: 'winner_model_a', 'winner_model_b', or 'winner_tie', along with the logits for each label.

Prompt: {data_point["prompt"]}  # プロンプトを表示します
Model A Response: {data_point["response_a"]}  # モデルAの応答を表示します
Model B Response: {data_point["response_b"]}  # モデルBの応答を表示します
"""

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

```

</div>
<div class="column-right">

# 日本語訳

```python
# このセルは空です。必要なコードをここに追加してください。
```

</div>
</details>

In [ ]:
# このセルは空です。必要なコードをここに追加してください。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
'''
MODEL_PATH = "/kaggle/input/llama-3/transformers/8b-hf/1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model1 = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map = "auto",
    trust_remote_code = True,
    quantization_config=quantization_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, dtype=torch.float16)
'''
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルとトークナイザーの設定を行います
'''
MODEL_PATH = "/kaggle/input/llama-3/transformers/8b-hf/1"  # 使用するモデルのパスを指定します

# 量子化設定を定義します
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4ビット量子化を使用
    bnb_4bit_quant_type="nf4",  # 量子化タイプを指定
    bnb_4bit_compute_dtype=torch.float16,  # 計算に使用するデータ型を指定
    bnb_4bit_use_double_quant=True,  # ダブル量子化を使用
)

# モデルを事前学習済みのパスから読み込む
model1 = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",  # 自動的にデバイスマッピングを設定
    trust_remote_code=True,  # リモートコードを信頼
    quantization_config=quantization_config,  # 先ほど定義した量子化設定を使用
)

# トークナイザーをモデルのパスから読み込む
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, dtype=torch.float16)  # データ型をfloat16に設定
'''
```

</div>
</details>

In [ ]:
# モデルとトークナイザーの設定を行います
'''
MODEL_PATH = "/kaggle/input/llama-3/transformers/8b-hf/1"  # 使用するモデルのパスを指定します

# 量子化設定を定義します
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4ビット量子化を使用
    bnb_4bit_quant_type="nf4",  # 量子化タイプを指定
    bnb_4bit_compute_dtype=torch.float16,  # 計算に使用するデータ型を指定
    bnb_4bit_use_double_quant=True,  # ダブル量子化を使用
)

# モデルを事前学習済みのパスから読み込む
model1 = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",  # 自動的にデバイスマッピングを設定
    trust_remote_code=True,  # リモートコードを信頼
    quantization_config=quantization_config,  # 先ほど定義した量子化設定を使用
)

# トークナイザーをモデルのパスから読み込む
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, dtype=torch.float16)  # データ型をfloat16に設定
'''

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.float16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",

    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit"
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/kaggle/input/unslothmistral-7b-instruct-v0-2-bnb-4bit/unsloth-mistral-7b-instruct-v0.2-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルの設定を行います
max_seq_length = 2048  # シーケンスの最大長を指定します。内部でRoPEスケーリングを自動サポートします。
dtype = torch.float16  # データ型を指定。自動検出にしない場合はfloat16を使用。Tesla T4やV100の場合はfloat16、Ampere+の場合はBfloat16を選択。
load_in_4bit = True  # メモリ使用量を抑えるために4ビット量子化を使用します。Falseを選択すると使用しません。

# 4ビット事前量子化モデルのリスト（ダウンロードが4倍速く、OOM（メモリ不足）を回避できます）
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",  # サポートされているモデル

    "unsloth/llama-2-7b-bnb-4bit",  # 他のモデルもここに追加できます
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit"
]  # さらに多くのモデルは https://huggingface.co/unsloth で確認できます

# FastLanguageModelを使用してモデルとトークナイザーを事前学習済みのものから読み込みます
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="/kaggle/input/unslothmistral-7b-instruct-v0-2-bnb-4bit/unsloth-mistral-7b-instruct-v0.2-bnb-4bit",  # 任意のモデルを選択
    max_seq_length=max_seq_length,  # 最大シーケンス長を指定
    dtype=dtype,  # データ型を指定
    load_in_4bit=load_in_4bit,  # 4ビット量子化を使用するかどうかを指定
    # token="hf_...",  # gatedモデルを使用する際に必要なトークン（必要に応じてコメントを解除）
)
```

</div>
</details>

In [ ]:
# モデルの設定を行います
max_seq_length = 2048  # シーケンスの最大長を指定します。内部でRoPEスケーリングを自動サポートします。
dtype = torch.float16  # データ型を指定。自動検出にしない場合はfloat16を使用。Tesla T4やV100の場合はfloat16、Ampere+の場合はBfloat16を選択。
load_in_4bit = True  # メモリ使用量を抑えるために4ビット量子化を使用します。Falseを選択すると使用しません。

# 4ビット事前量子化モデルのリスト（ダウンロードが4倍速く、OOM（メモリ不足）を回避できます）
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",  # サポートされているモデル

    "unsloth/llama-2-7b-bnb-4bit",  # 他のモデルもここに追加できます
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit"
]  # さらに多くのモデルは https://huggingface.co/unsloth で確認できます

# FastLanguageModelを使用してモデルとトークナイザーを事前学習済みのものから読み込みます
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="/kaggle/input/unslothmistral-7b-instruct-v0-2-bnb-4bit/unsloth-mistral-7b-instruct-v0.2-bnb-4bit",  # 任意のモデルを選択
    max_seq_length=max_seq_length,  # 最大シーケンス長を指定
    dtype=dtype,  # データ型を指定
    load_in_4bit=load_in_4bit,  # 4ビット量子化を使用するかどうかを指定
    # token="hf_...",  # gatedモデルを使用する際に必要なトークン（必要に応じてコメントを解除）
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
```

</div>
<div class="column-right">

# 日本語訳

```python
# PEFT（Parameter-Efficient Fine-Tuning）モデルを取得します
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # 任意の正の整数を指定します。推奨値は8, 16, 32, 64, 128です。
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",  # PEFTを適用するモジュールを指定
                    "gate_proj", "up_proj", "down_proj",],
    lora_alpha=16,  # LoRAのα値を指定
    lora_dropout=0,  # LoRAのドロップアウト率。任意の値を指定できますが、0が最適化されています。
    bias="none",  # バイアス設定。任意の値をサポートしますが、"none"が最適化されています。
    # [NEW] "unsloth"は30%少ないVRAMを使用し、2倍大きなバッチサイズに対応します！
    use_gradient_checkpointing="unsloth",  # 長いコンテキストのためにTrueまたは"unsloth"を指定
    random_state=3407,  # 乱数の状態を指定
    use_rslora=False,  # ランク安定LoRAを使用するかどうかを指定
    loftq_config=None,  # LoftQの設定を指定
)
```

</div>
</details>

In [ ]:
# PEFT（Parameter-Efficient Fine-Tuning）モデルを取得します
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # 任意の正の整数を指定します。推奨値は8, 16, 32, 64, 128です。
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",  # PEFTを適用するモジュールを指定
                    "gate_proj", "up_proj", "down_proj",],
    lora_alpha=16,  # LoRAのα値を指定
    lora_dropout=0,  # LoRAのドロップアウト率。任意の値を指定できますが、0が最適化されています。
    bias="none",  # バイアス設定。任意の値をサポートしますが、"none"が最適化されています。
    # [NEW] "unsloth"は30%少ないVRAMを使用し、2倍大きなバッチサイズに対応します！
    use_gradient_checkpointing="unsloth",  # 長いコンテキストのためにTrueまたは"unsloth"を指定
    random_state=3407,  # 乱数の状態を指定
    use_rslora=False,  # ランク安定LoRAを使用するかどうかを指定
    loftq_config=None,  # LoftQの設定を指定
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
submission = pd.read_csv(open('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv'))
test = pd.read_csv(open('/kaggle/input/lmsys-chatbot-arena/test.csv'))
train = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")
```

</div>
<div class="column-right">

# 日本語訳

```python
# 提出ファイルとデータセットを読み込みます
submission = pd.read_csv(open('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv'))  # サンプル提出ファイルを読み込み
test = pd.read_csv(open('/kaggle/input/lmsys-chatbot-arena/test.csv'))  # テストデータセットを読み込み
train = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")  # トレーニングデータセットを読み込み
```

</div>
</details>

In [ ]:
# 提出ファイルとデータセットを読み込みます
submission = pd.read_csv(open('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv'))  # サンプル提出ファイルを読み込み
test = pd.read_csv(open('/kaggle/input/lmsys-chatbot-arena/test.csv'))  # テストデータセットを読み込み
train = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")  # トレーニングデータセットを読み込み

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
result = []
for index, row in train.iterrows():
    if(row['winner_model_a']==1): result.append('winner_model_a')
    elif(row['winner_model_b']==1): result.append('winner_model_b')
    elif(row['winner_tie']==1): result.append('winner_tie')
train.insert(6, "Preference", result, True)
train
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニングデータセットにPreference列を追加します
result = []  # 結果を格納するリストを初期化
# トレーニングデータセットの各行を反復処理
for index, row in train.iterrows():
    # 各勝者モデルの情報に基づいて結果を追加
    if(row['winner_model_a'] == 1): 
        result.append('winner_model_a')  # モデルAが勝者の場合
    elif(row['winner_model_b'] == 1): 
        result.append('winner_model_b')  # モデルBが勝者の場合
    elif(row['winner_tie'] == 1): 
        result.append('winner_tie')  # 引き分けの場合

# トレーニングデータフレームの6列目に新しいPreference列を挿入
train.insert(6, "Preference", result, True)  
train  # 更新されたトレーニングデータフレームを表示
```

</div>
</details>

In [ ]:
# トレーニングデータセットにPreference列を追加します
result = []  # 結果を格納するリストを初期化
# トレーニングデータセットの各行を反復処理
for index, row in train.iterrows():
    # 各勝者モデルの情報に基づいて結果を追加
    if(row['winner_model_a'] == 1): 
        result.append('winner_model_a')  # モデルAが勝者の場合
    elif(row['winner_model_b'] == 1): 
        result.append('winner_model_b')  # モデルBが勝者の場合
    elif(row['winner_tie'] == 1): 
        result.append('winner_tie')  # 引き分けの場合

# トレーニングデータフレームの6列目に新しいPreference列を挿入
train.insert(6, "Preference", result, True)  
train  # 更新されたトレーニングデータフレームを表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train = train[:1000]
train.shape
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニングデータセットの最初の1000行を選択します
train = train[:1000]  # 先頭1000行を抽出
train.shape  # データフレームの形状を表示（行数と列数）
```

</div>
</details>

In [ ]:
# トレーニングデータセットの最初の1000行を選択します
train = train[:1000]  # 先頭1000行を抽出
train.shape  # データフレームの形状を表示（行数と列数）

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train_size = int(train.shape[0]*0.6)
test_val_size = train.shape[0]-train_size
train_, test_val  = train_test_split(train, train_size=train_size,
                                    test_size=test_val_size,
                                    random_state=42)

test_, val_ = train_test_split(test_val, train_size=int(test_val_size/2),
                                    test_size=int(test_val_size/2),
                                    random_state=42)
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニングデータとテストデータを分割します
train_size = int(train.shape[0] * 0.6)  # トレーニングデータのサイズを全体の60%に設定
test_val_size = train.shape[0] - train_size  # 残りのデータをテストとバリデーション用に設定

# トレーニングデータをトレーニングセットとテスト・バリデーションセットに分割
train_, test_val = train_test_split(train, train_size=train_size,
                                     test_size=test_val_size,
                                     random_state=42)  # ランダム状態を42に設定

# テスト・バリデーションセットをさらにテストセットとバリデーションセットに分割
test_, val_ = train_test_split(test_val, train_size=int(test_val_size / 2),
                                test_size=int(test_val_size / 2),
                                random_state=42)  # 同様にランダム状態を42に設定
```

</div>
</details>

In [ ]:
# トレーニングデータとテストデータを分割します
train_size = int(train.shape[0] * 0.6)  # トレーニングデータのサイズを全体の60%に設定
test_val_size = train.shape[0] - train_size  # 残りのデータをテストとバリデーション用に設定

# トレーニングデータをトレーニングセットとテスト・バリデーションセットに分割
train_, test_val = train_test_split(train, train_size=train_size,
                                     test_size=test_val_size,
                                     random_state=42)  # ランダム状態を42に設定

# テスト・バリデーションセットをさらにテストセットとバリデーションセットに分割
test_, val_ = train_test_split(test_val, train_size=int(test_val_size / 2),
                                test_size=int(test_val_size / 2),
                                random_state=42)  # 同様にランダム状態を42に設定

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def past_generate_prompt(data_point):
    return f"""
            Analyze the prompt and responses(response_a, response_b) from two chatbots(model_a, model_b).
            Then predict the human preference of those responses- if it is "winner_model_a", "winner_model_b" or
            "winner_tie". Return the answer as the correspoding preference label "winner_model_a", "winner_model_b" or
            "winner_tie" and the logits for each label in the order of preference.
            ----------------------------------------------------------------------------------------------------------
            prompt: {data_point["prompt"]}
            ----------------------------------------------------------------------------------------------------------
            model_a: {data_point["model_a"]}
            response_a: {data_point["response_a"]}
            ----------------------------------------------------------------------------------------------------------
            model_b: {data_point["model_b"]}
            response_b: {data_point["response_b"]}
            ----------------------------------------------------------------------------------------------------------
            Preference= {data_point["Preference"]} """.strip()

def generate_prompt(data_point):
    return f"""Analyze the conversation between two chatbots (model_a and model_b) and their corresponding responses (response_a and response_b) to a given prompt. Determine which model provided the more preferred
response based on the human preference label (Preference). Return the predicted preference as one of three labels: 'winner_model_a', 'winner_model_b', or 'winner_tie', along with the logits for each label.

Prompt: {data_point["prompt"]}
Model A Response: {data_point["response_a"]}
Model B Response: {data_point["response_b"]}
Human Preference Label: {data_point["Preference"]}
"""

def past_generate_test_prompt(data_point):
    return f"""
            Analyze the prompt and responses(response_a, response_b) from two chatbots(model_a, model_b).
            Then predict the human preference of those responses- if it is "winner_model_a", "winner_model_b" or
            "winner_tie". Return the answer as the correspoding preference label "winner_model_a", "winner_model_b" or
            "winner_tie" and the logits for each label in the order of preference.
            ----------------------------------------------------------------------------------------------------------
            prompt: {data_point["prompt"]}
            ----------------------------------------------------------------------------------------------------------

            response_a: {data_point["response_a"]}
            ----------------------------------------------------------------------------------------------------------

            response_b: {data_point["response_b"]}
            ----------------------------------------------------------------------------------------------------------
            Preference: """.strip()

def generate_test_prompt(data_point):
    return f"""Analyze the conversation between two chatbots (model_a and model_b) and their corresponding responses (response_a and response_b) to a given prompt. Determine which model provided the more preferred
response based on the human preference label (Preference). Return the predicted preference as one of three labels: 'winner_model_a', 'winner_model_b', or 'winner_tie', along with the logits for each label.

Prompt: {data_point["prompt"]}
Model A Response: {data_point["response_a"]}
Model B Response: {data_point["response_b"]}
"""

```

</div>
<div class="column-right">

# 日本語訳

```python
# プロンプトと応答を分析するための関数を定義します

# 過去のデータポイントからプロンプトを生成する関数
def past_generate_prompt(data_point):
    return f"""
            Analyze the prompt and responses(response_a, response_b) from two chatbots(model_a, model_b).
            Then predict the human preference of those responses- if it is "winner_model_a", "winner_model_b" or
            "winner_tie". Return the answer as the correspoding preference label "winner_model_a", "winner_model_b" or
            "winner_tie" and the logits for each label in the order of preference.
            ----------------------------------------------------------------------------------------------------------
            prompt: {data_point["prompt"]}  # プロンプトを表示
            ----------------------------------------------------------------------------------------------------------
            model_a: {data_point["model_a"]}  # モデルAの情報を表示
            response_a: {data_point["response_a"]}  # モデルAの応答を表示
            ----------------------------------------------------------------------------------------------------------
            model_b: {data_point["model_b"]}  # モデルBの情報を表示
            response_b: {data_point["response_b"]}  # モデルBの応答を表示
            ----------------------------------------------------------------------------------------------------------
            Preference= {data_point["Preference"]} """  # 人間の好みを表示

# 現在のデータポイントからプロンプトを生成する関数
def generate_prompt(data_point):
    return f"""Analyze the conversation between two chatbots (model_a and model_b) and their corresponding responses (response_a and response_b) to a given prompt. Determine which model provided the more preferred
response based on the human preference label (Preference). Return the predicted preference as one of three labels: 'winner_model_a', 'winner_model_b', or 'winner_tie', along with the logits for each label.

Prompt: {data_point["prompt"]}  # プロンプトを表示
Model A Response: {data_point["response_a"]}  # モデルAの応答を表示
Model B Response: {data_point["response_b"]}  # モデルBの応答を表示
Human Preference Label: {data_point["Preference"]}  # 人間の好みラベルを表示
"""

# テストデータポイントからプロンプトを生成する関数
def past_generate_test_prompt(data_point):
    return f"""
            Analyze the prompt and responses(response_a, response_b) from two chatbots(model_a, model_b).
            Then predict the human preference of those responses- if it is "winner_model_a", "winner_model_b" or
            "winner_tie". Return the answer as the correspoding preference label "winner_model_a", "winner_model_b" or
            "winner_tie" and the logits for each label in the order of preference.
            ----------------------------------------------------------------------------------------------------------
            prompt: {data_point["prompt"]}  # プロンプトを表示
            ----------------------------------------------------------------------------------------------------------

            response_a: {data_point["response_a"]}  # モデルAの応答を表示
            ----------------------------------------------------------------------------------------------------------

            response_b: {data_point["response_b"]}  # モデルBの応答を表示
            ----------------------------------------------------------------------------------------------------------
            Preference: """  # 人間の好みを表示

# 現在のテストデータポイントからプロンプトを生成する関数
def generate_test_prompt(data_point):
    return f"""Analyze the conversation between two chatbots (model_a and model_b) and their corresponding responses (response_a and response_b) to a given prompt. Determine which model provided the more preferred
response based on the human preference label (Preference). Return the predicted preference as one of three labels: 'winner_model_a', 'winner_model_b', or 'winner_tie', along with the logits for each label.

Prompt: {data_point["prompt"]}  # プロンプトを表示
Model A Response: {data_point["response_a"]}  # モデルAの応答を表示
Model B Response: {data_point["response_b"]}  # モデルBの応答を表示
"""
```

</div>
</details>

In [ ]:
# プロンプトと応答を分析するための関数を定義します

# 過去のデータポイントからプロンプトを生成する関数
def past_generate_prompt(data_point):
    return f"""
            Analyze the prompt and responses(response_a, response_b) from two chatbots(model_a, model_b).
            Then predict the human preference of those responses- if it is "winner_model_a", "winner_model_b" or
            "winner_tie". Return the answer as the correspoding preference label "winner_model_a", "winner_model_b" or
            "winner_tie" and the logits for each label in the order of preference.
            ----------------------------------------------------------------------------------------------------------
            prompt: {data_point["prompt"]}  # プロンプトを表示
            ----------------------------------------------------------------------------------------------------------
            model_a: {data_point["model_a"]}  # モデルAの情報を表示
            response_a: {data_point["response_a"]}  # モデルAの応答を表示
            ----------------------------------------------------------------------------------------------------------
            model_b: {data_point["model_b"]}  # モデルBの情報を表示
            response_b: {data_point["response_b"]}  # モデルBの応答を表示
            ----------------------------------------------------------------------------------------------------------
            Preference= {data_point["Preference"]} """  # 人間の好みを表示

# 現在のデータポイントからプロンプトを生成する関数
def generate_prompt(data_point):
    return f"""Analyze the conversation between two chatbots (model_a and model_b) and their corresponding responses (response_a and response_b) to a given prompt. Determine which model provided the more preferred
response based on the human preference label (Preference). Return the predicted preference as one of three labels: 'winner_model_a', 'winner_model_b', or 'winner_tie', along with the logits for each label.

Prompt: {data_point["prompt"]}  # プロンプトを表示
Model A Response: {data_point["response_a"]}  # モデルAの応答を表示
Model B Response: {data_point["response_b"]}  # モデルBの応答を表示
Human Preference Label: {data_point["Preference"]}  # 人間の好みラベルを表示
"""

# テストデータポイントからプロンプトを生成する関数
def past_generate_test_prompt(data_point):
    return f"""
            Analyze the prompt and responses(response_a, response_b) from two chatbots(model_a, model_b).
            Then predict the human preference of those responses- if it is "winner_model_a", "winner_model_b" or
            "winner_tie". Return the answer as the correspoding preference label "winner_model_a", "winner_model_b" or
            "winner_tie" and the logits for each label in the order of preference.
            ----------------------------------------------------------------------------------------------------------
            prompt: {data_point["prompt"]}  # プロンプトを表示
            ----------------------------------------------------------------------------------------------------------

            response_a: {data_point["response_a"]}  # モデルAの応答を表示
            ----------------------------------------------------------------------------------------------------------

            response_b: {data_point["response_b"]}  # モデルBの応答を表示
            ----------------------------------------------------------------------------------------------------------
            Preference: """  # 人間の好みを表示

# 現在のテストデータポイントからプロンプトを生成する関数
def generate_test_prompt(data_point):
    return f"""Analyze the conversation between two chatbots (model_a and model_b) and their corresponding responses (response_a and response_b) to a given prompt. Determine which model provided the more preferred
response based on the human preference label (Preference). Return the predicted preference as one of three labels: 'winner_model_a', 'winner_model_b', or 'winner_tie', along with the logits for each label.

Prompt: {data_point["prompt"]}  # プロンプトを表示
Model A Response: {data_point["response_a"]}  # モデルAの応答を表示
Model B Response: {data_point["response_b"]}  # モデルBの応答を表示
"""

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
X_train = pd.DataFrame(train_.apply(generate_prompt, axis=1),
                       columns=["text"])
X_eval = pd.DataFrame(val_.apply(generate_prompt, axis=1),
                      columns=["text"])
X_test = pd.DataFrame(test_.apply(generate_test_prompt, axis=1), columns=["text"])

y_true = test_.Preference

train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニング、評価、テストデータセットを生成します

# トレーニングデータセットのプロンプトを生成し、DataFrameに変換
X_train = pd.DataFrame(train_.apply(generate_prompt, axis=1), columns=["text"])
# 評価データセットのプロンプトを生成し、DataFrameに変換
X_eval = pd.DataFrame(val_.apply(generate_prompt, axis=1), columns=["text"])
# テストデータセットのプロンプトを生成し、DataFrameに変換
X_test = pd.DataFrame(test_.apply(generate_test_prompt, axis=1), columns=["text"])

# テストデータの真のラベルを取得
y_true = test_.Preference

# pandas DataFrameからデータセットを生成
train_data = Dataset.from_pandas(X_train)  # トレーニングデータをDatasetに変換
eval_data = Dataset.from_pandas(X_eval)    # 評価データをDatasetに変換
```

</div>
</details>

In [ ]:
# トレーニング、評価、テストデータセットを生成します

# トレーニングデータセットのプロンプトを生成し、DataFrameに変換
X_train = pd.DataFrame(train_.apply(generate_prompt, axis=1), columns=["text"])
# 評価データセットのプロンプトを生成し、DataFrameに変換
X_eval = pd.DataFrame(val_.apply(generate_prompt, axis=1), columns=["text"])
# テストデータセットのプロンプトを生成し、DataFrameに変換
X_test = pd.DataFrame(test_.apply(generate_test_prompt, axis=1), columns=["text"])

# テストデータの真のラベルを取得
y_true = test_.Preference

# pandas DataFrameからデータセットを生成
train_data = Dataset.from_pandas(X_train)  # トレーニングデータをDatasetに変換
eval_data = Dataset.from_pandas(X_eval)    # 評価データをDatasetに変換

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def evaluate(y_true, y_pred):
    labels = ['winner_model_a', 'winner_model_b', 'winner_tie']
    mapping = {'none':0, 'winner_model_a': 1, 'winner_model_b': 2, 'winner_tie': 3}
    def map_func(x):
        return mapping.get(x, 1)

    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルの評価を行う関数を定義します
def evaluate(y_true, y_pred):
    labels = ['winner_model_a', 'winner_model_b', 'winner_tie']  # ラベルを定義
    mapping = {'none': 0, 'winner_model_a': 1, 'winner_model_b': 2, 'winner_tie': 3}  # ラベルのマッピングを定義

    # ラベルを対応する数値にマッピングする関数
    def map_func(x):
        return mapping.get(x, 1)  # デフォルトは1（winner_model_a）

    # 真のラベルと予測ラベルをマッピングして変換
    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # 正確度を計算
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')  # 正確度を出力

    # 各ラベルの正確度を生成
    unique_labels = set(y_true)  # ユニークなラベルを取得

    for label in unique_labels:
        # 各ラベルのインデックスを取得
        label_indices = [i for i in range(len(y_true)) if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]  # 真のラベル
        label_y_pred = [y_pred[i] for i in label_indices]  # 予測ラベル
        accuracy = accuracy_score(label_y_true, label_y_pred)  # 各ラベルの正確度を計算
        print(f'Accuracy for label {label}: {accuracy:.3f}')  # 各ラベルの正確度を出力

    # 分類レポートを生成
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)  # 分類レポートを出力

    # 混同行列を生成
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)  # 混同行列を出力
```

</div>
</details>

In [ ]:
# モデルの評価を行う関数を定義します
def evaluate(y_true, y_pred):
    labels = ['winner_model_a', 'winner_model_b', 'winner_tie']  # ラベルを定義
    mapping = {'none': 0, 'winner_model_a': 1, 'winner_model_b': 2, 'winner_tie': 3}  # ラベルのマッピングを定義

    # ラベルを対応する数値にマッピングする関数
    def map_func(x):
        return mapping.get(x, 1)  # デフォルトは1（winner_model_a）

    # 真のラベルと予測ラベルをマッピングして変換
    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # 正確度を計算
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')  # 正確度を出力

    # 各ラベルの正確度を生成
    unique_labels = set(y_true)  # ユニークなラベルを取得

    for label in unique_labels:
        # 各ラベルのインデックスを取得
        label_indices = [i for i in range(len(y_true)) if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]  # 真のラベル
        label_y_pred = [y_pred[i] for i in label_indices]  # 予測ラベル
        accuracy = accuracy_score(label_y_true, label_y_pred)  # 各ラベルの正確度を計算
        print(f'Accuracy for label {label}: {accuracy:.3f}')  # 各ラベルの正確度を出力

    # 分類レポートを生成
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)  # 分類レポートを出力

    # 混同行列を生成
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)  # 混同行列を出力

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from trl import SFTTrainer
from transformers import TrainingArguments


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_data,
    eval_dataset = eval_data,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        #evaluation_strategy="steps",         # evaluate at the end of each epoch
        #save_strategy="steps",
        #save_steps=10,                      # save checkpoint every 500 steps
        #eval_steps=10,
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = True,
        bf16 = False,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        #load_best_model_at_end=True,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)
```

</div>
<div class="column-right">

# 日本語訳

```python
# SFT（Supervised Fine-Tuning）トレーナーをインポートします
from trl import SFTTrainer
from transformers import TrainingArguments

# SFTTrainerのインスタンスを作成します
trainer = SFTTrainer(
    model=model,  # 学習するモデル
    tokenizer=tokenizer,  # 使用するトークナイザー
    train_dataset=train_data,  # トレーニングデータセット
    eval_dataset=eval_data,  # 評価データセット
    dataset_text_field="text",  # テキストフィールド
    max_seq_length=max_seq_length,  # 最大シーケンス長
    dataset_num_proc=2,  # データセットの処理を並列で行うプロセス数
    packing=False,  # 短いシーケンスのトレーニングを5倍速くすることができます
    args=TrainingArguments(
        per_device_train_batch_size=1,  # デバイスあたりのトレーニングバッチサイズ
        gradient_accumulation_steps=2,  # 勾配蓄積ステップ
        warmup_steps=5,  # ウォームアップステップ
        #evaluation_strategy="steps",  # 各エポックの終わりに評価
        #save_strategy="steps",  # チェックポイントを保存する戦略
        #save_steps=10,  # チェックポイントを500ステップごとに保存
        #eval_steps=10,
        max_steps=100,  # 最大ステップ数
        learning_rate=2e-4,  # 学習率
        fp16=True,  # フロート16精度を使用
        bf16=False,  # Bfloat16精度を使用しない
        logging_steps=1,  # ログ出力のステップ数
        optim="adamw_8bit",  # オプティマイザー
        weight_decay=0.01,  # 重みの減衰
        #load_best_model_at_end=True,  # 最良モデルを最後に読み込む
        lr_scheduler_type="linear",  # 学習率スケジューラーのタイプ
        seed=3407,  # シード値
        output_dir="outputs",  # 出力ディレクトリ
    ),
)
```

</div>
</details>

In [ ]:
# SFT（Supervised Fine-Tuning）トレーナーをインポートします
from trl import SFTTrainer
from transformers import TrainingArguments

# SFTTrainerのインスタンスを作成します
trainer = SFTTrainer(
    model=model,  # 学習するモデル
    tokenizer=tokenizer,  # 使用するトークナイザー
    train_dataset=train_data,  # トレーニングデータセット
    eval_dataset=eval_data,  # 評価データセット
    dataset_text_field="text",  # テキストフィールド
    max_seq_length=max_seq_length,  # 最大シーケンス長
    dataset_num_proc=2,  # データセットの処理を並列で行うプロセス数
    packing=False,  # 短いシーケンスのトレーニングを5倍速くすることができます
    args=TrainingArguments(
        per_device_train_batch_size=1,  # デバイスあたりのトレーニングバッチサイズ
        gradient_accumulation_steps=2,  # 勾配蓄積ステップ
        warmup_steps=5,  # ウォームアップステップ
        #evaluation_strategy="steps",  # 各エポックの終わりに評価
        #save_strategy="steps",  # チェックポイントを保存する戦略
        #save_steps=10,  # チェックポイントを500ステップごとに保存
        #eval_steps=10,
        max_steps=100,  # 最大ステップ数
        learning_rate=2e-4,  # 学習率
        fp16=True,  # フロート16精度を使用
        bf16=False,  # Bfloat16精度を使用しない
        logging_steps=1,  # ログ出力のステップ数
        optim="adamw_8bit",  # オプティマイザー
        weight_decay=0.01,  # 重みの減衰
        #load_best_model_at_end=True,  # 最良モデルを最後に読み込む
        lr_scheduler_type="linear",  # 学習率スケジューラーのタイプ
        seed=3407,  # シード値
        output_dir="outputs",  # 出力ディレクトリ
    ),
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import wandb
wandb.init(mode='disabled')
```

</div>
<div class="column-right">

# 日本語訳

```python
# W&B（Weights & Biases）をインポートし、初期化します
import wandb
wandb.init(mode='disabled')  # W&Bのモードを無効にして初期化します（ログを送信しない）
```

</div>
</details>

In [ ]:
# W&B（Weights & Biases）をインポートし、初期化します
import wandb
wandb.init(mode='disabled')  # W&Bのモードを無効にして初期化します（ログを送信しない）

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import gc
gc.collect()
```

</div>
<div class="column-right">

# 日本語訳

```python
# ガーベジコレクションを実行します
import gc  # ガーベジコレクションモジュールをインポート
gc.collect()  # 使用されていないメモリを解放するためにガーベジコレクションを実行
```

</div>
</details>

In [ ]:
# ガーベジコレクションを実行します
import gc  # ガーベジコレクションモジュールをインポート
gc.collect()  # 使用されていないメモリを解放するためにガーベジコレクションを実行

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
trainer.train()
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルのトレーニングを開始します
trainer.train()  # 定義したtrainerオブジェクトを使用してトレーニングを実行
```

</div>
</details>

In [ ]:
# モデルのトレーニングを開始します
trainer.train()  # 定義したtrainerオブジェクトを使用してトレーニングを実行

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

```

</div>
<div class="column-right">

# 日本語訳

```python
# このセルは空です。必要なコードをここに追加してください。
```

</div>
</details>

In [ ]:
# このセルは空です。必要なコードをここに追加してください。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

```

</div>
<div class="column-right">

# 日本語訳

```python
# このセルは空です。必要なコードをここに追加してください。
```

</div>
</details>

In [ ]:
# このセルは空です。必要なコードをここに追加してください。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
test['text'] = test.apply(lambda x: generate_test_prompt(x), axis=1)
```

</div>
<div class="column-right">

# 日本語訳

```python
# テストデータにプロンプトを生成するテキスト列を追加します
test['text'] = test.apply(lambda x: generate_test_prompt(x), axis=1)  # 各テストデータポイントに対してプロンプトを生成し、'text'列に追加
```

</div>
</details>

In [ ]:
# テストデータにプロンプトを生成するテキスト列を追加します
test['text'] = test.apply(lambda x: generate_test_prompt(x), axis=1)  # 各テストデータポイントに対してプロンプトを生成し、'text'列に追加

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
'''
prompt = test.loc[2]["text"]

#print(prompt)

#FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(prompt, return_tensors='pt').to(device)

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
out1 = tokenizer.batch_decode(outputs)
print("*************")
print(out1)
'''
```

</div>
<div class="column-right">

# 日本語訳

```python
# テストデータの特定のプロンプトを使用してモデルを推論します
'''
prompt = test.loc[2]["text"]  # テストデータのインデックス2からプロンプトを取得

#print(prompt)  # プロンプトを表示するためのコメントアウト

#FastLanguageModel.for_inference(model)  # ネイティブ2倍速の推論を有効にします

# トークナイザーを使用してプロンプトを入力テンソルに変換し、指定したデバイスに移動
inputs = tokenizer(prompt, return_tensors='pt').to(device)

# モデルを使用して出力を生成
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)  # 最大64の新しいトークンを生成
out1 = tokenizer.batch_decode(outputs)  # 出力をデコード
print("*************")  # 出力のセパレータ
print(out1)  # 出力を表示
'''
```

</div>
</details>

In [ ]:
# テストデータの特定のプロンプトを使用してモデルを推論します
'''
prompt = test.loc[2]["text"]  # テストデータのインデックス2からプロンプトを取得

#print(prompt)  # プロンプトを表示するためのコメントアウト

#FastLanguageModel.for_inference(model)  # ネイティブ2倍速の推論を有効にします

# トークナイザーを使用してプロンプトを入力テンソルに変換し、指定したデバイスに移動
inputs = tokenizer(prompt, return_tensors='pt').to(device)

# モデルを使用して出力を生成
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)  # 最大64の新しいトークンを生成
out1 = tokenizer.batch_decode(outputs)  # 出力をデコード
print("*************")  # 出力のセパレータ
print(out1)  # 出力を表示
'''

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def extract_logits(text):
    try:
        d = {'winner_a': [], 'winner_b': [], 'winner_tie': []}
        logits = re.findall(r"Logits: \[(.*?)\]", text)[0]
        logits_dict = {k.strip(): float(v) for k, v in [item.split(": ") for item in logits[1:-1].split(", ")]}

        for k, v in logits_dict.items():
            if '_a' in k:
                d['winner_a'].append(v)
            elif '_b' in k:
                d['winner_b'].append(v)
            elif '_tie' in k:
                d['winner_tie'].append(v)
            
        df = pd.DataFrame.from_dict(d)
    
    except:
        d = {'winner_a': [0.33], 'winner_b': [0.33], 'winner_tie': [0.33]}
        df = pd.DataFrame.from_dict(d)


    
    

    return df
```

</div>
<div class="column-right">

# 日本語訳

```python
# テキストからロジットを抽出する関数を定義します
def extract_logits(text):
    try:
        d = {'winner_a': [], 'winner_b': [], 'winner_tie': []}  # 各クラスのロジットを格納する辞書を初期化
        # テキストからロジットの部分を抽出
        logits = re.findall(r"Logits: \[(.*?)\]", text)[0]  # ロジットの情報を正規表現で取得
        # ロジットの辞書を生成
        logits_dict = {k.strip(): float(v) for k, v in [item.split(": ") for item in logits[1:-1].split(", ")]}

        # ロジットを各クラスに分類
        for k, v in logits_dict.items():
            if '_a' in k:
                d['winner_a'].append(v)  # モデルAのロジットを追加
            elif '_b' in k:
                d['winner_b'].append(v)  # モデルBのロジットを追加
            elif '_tie' in k:
                d['winner_tie'].append(v)  # 引き分けのロジットを追加
            
        df = pd.DataFrame.from_dict(d)  # 辞書からDataFrameを作成
    
    except:
        d = {'winner_a': [0.33], 'winner_b': [0.33], 'winner_tie': [0.33]}  # エラー時のデフォルト値を設定
        df = pd.DataFrame.from_dict(d)  # デフォルトのDataFrameを作成

    return df  # データフレームを返す
```

</div>
</details>

In [ ]:
# テキストからロジットを抽出する関数を定義します
def extract_logits(text):
    try:
        d = {'winner_a': [], 'winner_b': [], 'winner_tie': []}  # 各クラスのロジットを格納する辞書を初期化
        # テキストからロジットの部分を抽出
        logits = re.findall(r"Logits: \[(.*?)\]", text)[0]  # ロジットの情報を正規表現で取得
        # ロジットの辞書を生成
        logits_dict = {k.strip(): float(v) for k, v in [item.split(": ") for item in logits[1:-1].split(", ")]}

        # ロジットを各クラスに分類
        for k, v in logits_dict.items():
            if '_a' in k:
                d['winner_a'].append(v)  # モデルAのロジットを追加
            elif '_b' in k:
                d['winner_b'].append(v)  # モデルBのロジットを追加
            elif '_tie' in k:
                d['winner_tie'].append(v)  # 引き分けのロジットを追加
            
        df = pd.DataFrame.from_dict(d)  # 辞書からDataFrameを作成
    
    except:
        d = {'winner_a': [0.33], 'winner_b': [0.33], 'winner_tie': [0.33]}  # エラー時のデフォルト値を設定
        df = pd.DataFrame.from_dict(d)  # デフォルトのDataFrameを作成

    return df  # データフレームを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
prompt = X_test.iloc[2]["text"]

#print(prompt)



FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(prompt, return_tensors='pt').to("cuda")

#print(inputs)

print("*************")
outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
out1 = tokenizer.batch_decode(outputs)

print(out1)


```

</div>
<div class="column-right">

# 日本語訳

```python
# テストデータの特定のプロンプトを使用してモデルを推論します
prompt = X_test.iloc[2]["text"]  # テストデータのインデックス2からプロンプトを取得

#print(prompt)  # プロンプトを表示するためのコメントアウト

FastLanguageModel.for_inference(model)  # ネイティブ2倍速の推論を有効にします

# トークナイザーを使用してプロンプトを入力テンソルに変換し、CUDAデバイスに移動
inputs = tokenizer(prompt, return_tensors='pt').to("cuda")

#print(inputs)  # 入力テンソルを表示するためのコメントアウト

print("*************")  # 出力のセパレータ
# モデルを使用して出力を生成
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)  # 最大64の新しいトークンを生成
out1 = tokenizer.batch_decode(outputs)  # 出力をデコード

print(out1)  # 出力を表示
```

</div>
</details>

In [ ]:
# テストデータの特定のプロンプトを使用してモデルを推論します
prompt = X_test.iloc[2]["text"]  # テストデータのインデックス2からプロンプトを取得

#print(prompt)  # プロンプトを表示するためのコメントアウト

FastLanguageModel.for_inference(model)  # ネイティブ2倍速の推論を有効にします

# トークナイザーを使用してプロンプトを入力テンソルに変換し、CUDAデバイスに移動
inputs = tokenizer(prompt, return_tensors='pt').to("cuda")

#print(inputs)  # 入力テンソルを表示するためのコメントアウト

print("*************")  # 出力のセパレータ
# モデルを使用して出力を生成
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)  # 最大64の新しいトークンを生成
out1 = tokenizer.batch_decode(outputs)  # 出力をデコード

print(out1)  # 出力を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

```

</div>
<div class="column-right">

# 日本語訳

```python
# このセルは空です。必要なコードをここに追加してください。
```

</div>
</details>

In [ ]:
# このセルは空です。必要なコードをここに追加してください。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# import re
# label_pattern = re.compile(r'Human Preference Label:\s*(\w+)')
```

</div>
<div class="column-right">

# 日本語訳

```python
# 正規表現をインポートし、人間の好みラベルを抽出するためのパターンを定義します
# import re  # 正規表現モジュールをインポート
# label_pattern = re.compile(r'Human Preference Label:\s*(\w+)')  # 人間の好みラベルを抽出するための正規表現パターン
```

</div>
</details>

In [ ]:
# 正規表現をインポートし、人間の好みラベルを抽出するためのパターンを定義します
# import re  # 正規表現モジュールをインポート
# label_pattern = re.compile(r'Human Preference Label:\s*(\w+)')  # 人間の好みラベルを抽出するための正規表現パターン

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# def get_logits(text):
#     logits_pattern = re.compile(r'Logits:\s*\[(.*?)\]')
#     logits_match = logits_pattern.search(text[0])

#     if logits_match:
#         logits_str = logits_match.group(1)
#         logits_list = [(x.strip()) for x in logits_str.split(',')]

#         # Output results
#         #print("Logits:", logits_list)
#     else:
#         # if it is not able to find string, give deafult values
#         logits_list = ["'winner_model_a': 0.33", "'winner_model_b': 0.33", "'winner_tie': 0.33"]
    
    
    
#     return logits_list
```

</div>
<div class="column-right">

# 日本語訳

```python
# テキストからロジットを取得する関数を定義します
# def get_logits(text):
#     logits_pattern = re.compile(r'Logits:\s*\[(.*?)\]')  # ロジットを抽出するための正規表現パターンを定義
#     logits_match = logits_pattern.search(text[0])  # テキストからロジットを検索

#     if logits_match:
#         logits_str = logits_match.group(1)  # 見つかったロジットの文字列を取得
#         logits_list = [(x.strip()) for x in logits_str.split(',')]  # カンマで分割してリストに変換

#         # 結果を出力
#         #print("Logits:", logits_list)  # ロジットを表示（コメントアウトされています）
#     else:
#         # 一致する文字列が見つからなかった場合、デフォルト値を指定
#         logits_list = ["'winner_model_a': 0.33", "'winner_model_b': 0.33", "'winner_tie': 0.33"]
    
#     return logits_list  # ロジットのリストを返す
```

</div>
</details>

In [ ]:
# テキストからロジットを取得する関数を定義します
# def get_logits(text):
#     logits_pattern = re.compile(r'Logits:\s*\[(.*?)\]')  # ロジットを抽出するための正規表現パターンを定義
#     logits_match = logits_pattern.search(text[0])  # テキストからロジットを検索

#     if logits_match:
#         logits_str = logits_match.group(1)  # 見つかったロジットの文字列を取得
#         logits_list = [(x.strip()) for x in logits_str.split(',')]  # カンマで分割してリストに変換

#         # 結果を出力
#         #print("Logits:", logits_list)  # ロジットを表示（コメントアウトされています）
#     else:
#         # 一致する文字列が見つからなかった場合、デフォルト値を指定
#         logits_list = ["'winner_model_a': 0.33", "'winner_model_b': 0.33", "'winner_tie': 0.33"]
    
#     return logits_list  # ロジットのリストを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# y_pred = []
# logits = pd.DataFrame()
# for i in tqdm(range(len(X_test))):
#     prompt = X_test.iloc[i]["text"]
#     FastLanguageModel.for_inference(model) # Enable native 2x faster inference

#     inputs = tokenizer(prompt, return_tensors='pt').to("cuda")

#     outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
#     out1 = tokenizer.batch_decode(outputs)
#     try:
#         preferences = label_pattern.search(out1[0]).group(1)
#         lgts = extract_logits(out1[0])
#     except Exception as e:
#         print(out1)
#         preferences = 'winner_tie'
#     print(preferences)
#     print(lgts)
#     y_pred.append(preferences)
    
#     logits = pd.concat([logits, lgts],axis=0)
```

</div>
<div class="column-right">

# 日本語訳

```python
# テストデータに対する推論を実行し、予測とロジットを取得するループを定義します
# y_pred = []  # 予測結果を格納するリストを初期化
# logits = pd.DataFrame()  # ロジットを格納するデータフレームを初期化

# for i in tqdm(range(len(X_test))):  # テストデータの各インデックスに対してループ
#     prompt = X_test.iloc[i]["text"]  # プロンプトを取得
#     FastLanguageModel.for_inference(model)  # ネイティブ2倍速の推論を有効にします

#     # トークナイザーを使用してプロンプトを入力テンソルに変換し、CUDAデバイスに移動
#     inputs = tokenizer(prompt, return_tensors='pt').to("cuda")

#     # モデルを使用して出力を生成
#     outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
#     out1 = tokenizer.batch_decode(outputs)  # 出力をデコード

#     try:
#         preferences = label_pattern.search(out1[0]).group(1)  # 人間の好みラベルを抽出
#         lgts = extract_logits(out1[0])  # ロジットを抽出
#     except Exception as e:
#         print(out1)  # 出力を表示（例外が発生した場合）
#         preferences = 'winner_tie'  # デフォルトの好みラベルを設定

#     print(preferences)  # 予測された好みを表示
#     print(lgts)  # 抽出されたロジットを表示
#     y_pred.append(preferences)  # 予測結果をリストに追加
    
#     logits = pd.concat([logits, lgts], axis=0)  # ロジットをデータフレームに追加
```

</div>
</details>

In [ ]:
# テストデータに対する推論を実行し、予測とロジットを取得するループを定義します
# y_pred = []  # 予測結果を格納するリストを初期化
# logits = pd.DataFrame()  # ロジットを格納するデータフレームを初期化

# for i in tqdm(range(len(X_test))):  # テストデータの各インデックスに対してループ
#     prompt = X_test.iloc[i]["text"]  # プロンプトを取得
#     FastLanguageModel.for_inference(model)  # ネイティブ2倍速の推論を有効にします

#     # トークナイザーを使用してプロンプトを入力テンソルに変換し、CUDAデバイスに移動
#     inputs = tokenizer(prompt, return_tensors='pt').to("cuda")

#     # モデルを使用して出力を生成
#     outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
#     out1 = tokenizer.batch_decode(outputs)  # 出力をデコード

#     try:
#         preferences = label_pattern.search(out1[0]).group(1)  # 人間の好みラベルを抽出
#         lgts = extract_logits(out1[0])  # ロジットを抽出
#     except Exception as e:
#         print(out1)  # 出力を表示（例外が発生した場合）
#         preferences = 'winner_tie'  # デフォルトの好みラベルを設定

#     print(preferences)  # 予測された好みを表示
#     print(lgts)  # 抽出されたロジットを表示
#     y_pred.append(preferences)  # 予測結果をリストに追加
    
#     logits = pd.concat([logits, lgts], axis=0)  # ロジットをデータフレームに追加

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
##evaluate(y_true, y_pred)
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルの予測結果を評価します
##evaluate(y_true, y_pred)  # 真のラベルと予測結果を使って評価関数を呼び出す（コメントアウトされています）
```

</div>
</details>

In [ ]:
# モデルの予測結果を評価します
##evaluate(y_true, y_pred)  # 真のラベルと予測結果を使って評価関数を呼び出す（コメントアウトされています）

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# def get_logits(text):
#     logits_pattern = re.compile(r'Logits:\s*\[(.*?)\]')
#     logits_match = logits_pattern.search(text[0])

#     if logits_match:
#         logits_str = logits_match.group(1)
#         logits_list = [(x.strip()) for x in logits_str.split(',')]

#         # Output results
#         #print("Logits:", logits_list)
#     else:
#         # if it is not able to find string, give deafult values
#         logits_list = ["'winner_model_a': 0.33", "'winner_model_b': 0.33", "'winner_tie': 0.33"]
    
    
    
#     return logits_list
```

</div>
<div class="column-right">

# 日本語訳

```python
# テキストからロジットを取得する関数を定義します
# def get_logits(text):
#     logits_pattern = re.compile(r'Logits:\s*\[(.*?)\]')  # ロジットを抽出するための正規表現パターンを定義
#     logits_match = logits_pattern.search(text[0])  # テキストからロジットを検索

#     if logits_match:
#         logits_str = logits_match.group(1)  # 見つかったロジットの文字列を取得
#         logits_list = [(x.strip()) for x in logits_str.split(',')]  # カンマで分割してリストに変換

#         # 結果を出力
#         #print("Logits:", logits_list)  # ロジットを表示（コメントアウトされています）
#     else:
#         # 一致する文字列が見つからなかった場合、デフォルト値を指定
#         logits_list = ["'winner_model_a': 0.33", "'winner_model_b': 0.33", "'winner_tie': 0.33"]
    
#     return logits_list  # ロジットのリストを返す
```

</div>
</details>

In [ ]:
# テキストからロジットを取得する関数を定義します
# def get_logits(text):
#     logits_pattern = re.compile(r'Logits:\s*\[(.*?)\]')  # ロジットを抽出するための正規表現パターンを定義
#     logits_match = logits_pattern.search(text[0])  # テキストからロジットを検索

#     if logits_match:
#         logits_str = logits_match.group(1)  # 見つかったロジットの文字列を取得
#         logits_list = [(x.strip()) for x in logits_str.split(',')]  # カンマで分割してリストに変換

#         # 結果を出力
#         #print("Logits:", logits_list)  # ロジットを表示（コメントアウトされています）
#     else:
#         # 一致する文字列が見つからなかった場合、デフォルト値を指定
#         logits_list = ["'winner_model_a': 0.33", "'winner_model_b': 0.33", "'winner_tie': 0.33"]
    
#     return logits_list  # ロジットのリストを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
logits_list = pd.DataFrame()
for i in tqdm(range(len(test))):
    prompt = test.iloc[i]["text"]
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

    inputs = tokenizer(prompt, return_tensors='pt').to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    out1 = tokenizer.batch_decode(outputs)
    preferences = extract_logits(out1[0])
    #print(preferences)
    logits_list = pd.concat([logits_list, preferences],axis=0)
```

</div>
<div class="column-right">

# 日本語訳

```python
# テストデータに対する推論を実行し、ロジットのリストを取得します
logits_list = pd.DataFrame()  # ロジットを格納するためのデータフレームを初期化
for i in tqdm(range(len(test))):  # テストデータの各インデックスに対してループ
    prompt = test.iloc[i]["text"]  # プロンプトを取得
    FastLanguageModel.for_inference(model)  # ネイティブ2倍速の推論を有効にします

    # トークナイザーを使用してプロンプトを入力テンソルに変換し、CUDAデバイスに移動
    inputs = tokenizer(prompt, return_tensors='pt').to("cuda")

    # モデルを使用して出力を生成
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
    out1 = tokenizer.batch_decode(outputs)  # 出力をデコード
    preferences = extract_logits(out1[0])  # ロジットを抽出
    #print(preferences)  # 抽出したロジットを表示（コメントアウトされています）
    
    # ロジットをデータフレームに追加
    logits_list = pd.concat([logits_list, preferences], axis=0)  # ロジットのデータフレームを結合
```

</div>
</details>

In [ ]:
# テストデータに対する推論を実行し、ロジットのリストを取得します
logits_list = pd.DataFrame()  # ロジットを格納するためのデータフレームを初期化
for i in tqdm(range(len(test))):  # テストデータの各インデックスに対してループ
    prompt = test.iloc[i]["text"]  # プロンプトを取得
    FastLanguageModel.for_inference(model)  # ネイティブ2倍速の推論を有効にします

    # トークナイザーを使用してプロンプトを入力テンソルに変換し、CUDAデバイスに移動
    inputs = tokenizer(prompt, return_tensors='pt').to("cuda")

    # モデルを使用して出力を生成
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
    out1 = tokenizer.batch_decode(outputs)  # 出力をデコード
    preferences = extract_logits(out1[0])  # ロジットを抽出
    #print(preferences)  # 抽出したロジットを表示（コメントアウトされています）
    
    # ロジットをデータフレームに追加
    logits_list = pd.concat([logits_list, preferences], axis=0)  # ロジットのデータフレームを結合

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# import pandas as pd



# # Function to convert string list to dictionary
# def logits_to_dict(logits):
#     return {item.split(':')[0].strip("' "): float(item.split(':')[1].strip()) for item in logits}

# # Convert each list to a dictionary
# logits_dicts = [logits_to_dict(logits) for logits in logits_list]

# # Create DataFrame
# df = pd.DataFrame(logits_dicts)

# df

```

</div>
<div class="column-right">

# 日本語訳

```python
# pandasライブラリをインポートします
# import pandas as pd

# ロジットの文字列リストを辞書に変換する関数を定義します
# def logits_to_dict(logits):
#     return {item.split(':')[0].strip("' "): float(item.split(':')[1].strip()) for item in logits}  # 項目を分割して辞書を生成

# 各ロジットリストを辞書に変換します
# logits_dicts = [logits_to_dict(logits) for logits in logits_list]  # ロジットリストから辞書のリストを作成

# データフレームを作成します
# df = pd.DataFrame(logits_dicts)  # 辞書のリストからデータフレームを生成

# df  # データフレームを表示（コメントアウトされています）
```

</div>
</details>

In [ ]:
# pandasライブラリをインポートします
# import pandas as pd

# ロジットの文字列リストを辞書に変換する関数を定義します
# def logits_to_dict(logits):
#     return {item.split(':')[0].strip("' "): float(item.split(':')[1].strip()) for item in logits}  # 項目を分割して辞書を生成

# 各ロジットリストを辞書に変換します
# logits_dicts = [logits_to_dict(logits) for logits in logits_list]  # ロジットリストから辞書のリストを作成

# データフレームを作成します
# df = pd.DataFrame(logits_dicts)  # 辞書のリストからデータフレームを生成

# df  # データフレームを表示（コメントアウトされています）

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
df1 = logits_list.reset_index(drop=True)
```

</div>
<div class="column-right">

# 日本語訳

```python
# ロジットリストのインデックスをリセットして新しいデータフレームを作成します
df1 = logits_list.reset_index(drop=True)  # インデックスをリセットし、古いインデックスを削除
```

</div>
</details>

In [ ]:
# ロジットリストのインデックスをリセットして新しいデータフレームを作成します
df1 = logits_list.reset_index(drop=True)  # インデックスをリセットし、古いインデックスを削除

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
df = pd.concat([test['id'], df1],axis=1)
```

</div>
<div class="column-right">

# 日本語訳

```python
# テストデータの'id'列とロジットのデータフレームを結合します
df = pd.concat([test['id'], df1], axis=1)  # 'id'列を左側に、ロジットのデータを右側に結合
```

</div>
</details>

In [ ]:
# テストデータの'id'列とロジットのデータフレームを結合します
df = pd.concat([test['id'], df1], axis=1)  # 'id'列を左側に、ロジットのデータを右側に結合

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

```

</div>
<div class="column-right">

# 日本語訳

```python
# このセルは空です。必要なコードをここに追加してください。
```

</div>
</details>

In [ ]:
# このセルは空です。必要なコードをここに追加してください。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
df
```

</div>
<div class="column-right">

# 日本語訳

```python
# 結合されたデータフレームを表示します
df  # テストデータの'id'列とロジットデータを含むデータフレームを表示
```

</div>
</details>

In [ ]:
# 結合されたデータフレームを表示します
df  # テストデータの'id'列とロジットデータを含むデータフレームを表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
df.to_csv('submission.csv', index=False)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 結合されたデータフレームをCSVファイルとして保存します
df.to_csv('submission.csv', index=False)  # インデックスなしで'submission.csv'として保存
```

</div>
</details>

In [ ]:
# 結合されたデータフレームをCSVファイルとして保存します
df.to_csv('submission.csv', index=False)  # インデックスなしで'submission.csv'として保存

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

```

</div>
<div class="column-right">

# 日本語訳

```python
# このセルは空です。必要なコードをここに追加してください。
```

</div>
</details>

In [ ]:
# このセルは空です。必要なコードをここに追加してください。